In [21]:
using JuMP, GLPK
using DataFrames
using CSV
using PrettyTables
using Random

In [22]:
scenarios_df = CSV.read("load_profiles.csv", DataFrame)

n_scenarios = size(scenarios_df, 2)
M = 1000000000

1000000000

In [23]:
scenarios_df

Row,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,⋯
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,⋯
1,363.0,369.0,332.0,369.0,368.0,356.0,329.0,346.0,364.0,337.0,356.0,358.0,349.0,359.0,333.0,333.0,344.0,357.0,371.0,328.0,356.0,358.0,362.0,348.0,356.0,348.0,373.0,371.0,335.0,373.0,345.0,364.0,342.0,356.0,341.0,365.0,365.0,359.0,327.0,371.0,335.0,358.0,370.0,369.0,337.0,328.0,358.0,356.0,360.0,333.0,333.0,329.0,331.0,360.0,338.0,325.0,367.0,357.0,326.0,339.0,369.0,359.0,334.0,334.0,355.0,344.0,349.0,332.0,363.0,337.0,339.0,370.0,344.0,361.0,346.0,335.0,335.0,331.0,335.0,359.0,327.0,337.0,354.0,364.0,342.0,353.0,373.0,348.0,363.0,334.0,365.0,352.0,369.0,366.0,327.0,360.0,352.0,325.0,352.0,358.0,⋯
2,362.873,371.707,341.07,372.077,372.342,350.056,331.784,352.222,372.183,336.395,359.829,376.013,355.41,382.348,331.399,326.706,344.378,352.773,363.962,334.726,353.55,369.268,360.286,347.346,359.602,338.128,379.07,365.952,346.111,373.361,335.864,371.074,344.292,362.867,336.359,369.079,368.429,369.337,328.472,361.126,338.071,369.208,382.088,380.637,349.534,318.907,364.825,362.668,373.263,341.107,343.705,314.32,332.575,364.586,331.323,310.99,367.337,348.054,335.896,340.92,378.672,360.298,331.58,343.031,363.434,340.829,358.423,321.528,381.414,334.363,352.611,364.298,337.247,380.242,343.696,347.255,340.166,344.221,341.298,356.446,322.549,336.601,356.835,368.392,342.877,365.04,371.009,354.039,364.467,336.216,370.748,351.312,371.334,363.893,332.418,362.483,373.763,330.607,353.792,368.235,⋯
3,370.739,373.917,348.105,377.335,379.261,357.564,328.254,353.205,373.249,349.148,357.598,365.766,358.575,381.714,329.131,334.157,348.34,348.472,365.996,320.588,360.726,379.489,354.828,347.298,355.332,333.997,375.279,373.93,344.629,373.94,343.765,378.06,336.197,367.376,339.694,391.414,375.36,371.055,342.021,351.082,337.738,367.922,374.474,377.332,346.101,329.367,365.404,344.641,384.101,342.056,340.559,297.202,329.736,373.044,321.905,307.254,361.589,361.106,347.449,347.814,382.938,353.207,321.127,343.471,368.798,344.618,358.351,322.231,362.763,318.078,348.982,354.081,335.187,377.752,349.851,348.46,341.772,338.995,331.729,352.483,315.967,339.684,361.797,374.828,343.522,369.221,369.095,351.563,355.657,333.466,379.68,353.661,362.088,364.783,333.829,364.207,375.876,329.226,355.387,367.695,⋯
4,370.983,369.752,357.481,378.446,383.813,358.677,335.914,348.906,376.291,362.307,344.747,362.47,363.843,395.938,330.335,318.186,351.619,348.432,365.097,318.194,368.488,382.035,363.67,363.015,363.21,341.265,376.712,377.407,336.619,376.3,350.577,382.742,337.117,385.985,350.685,388.346,382.198,380.113,346.064,355.28,332.308,370.396,378.105,385.957,348.441,323.528,367.369,361.16,382.601,337.223,332.562,300.384,317.951,393.658,320.511,305.114,357.344,359.773,350.393,357.971,396.409,367.749,322.97,349.48,361.68,357.166,347.411,321.765,375.69,319.369,357.594,355.989,341.683,381.137,349.615,356.427,347.451,352.446,330.436,358.533,321.437,326.931,363.807,380.182,345.569,375.962,389.544,346.575,370.535,34

In [24]:
in_sample_n = 10
q =0.9*in_sample_n

9.0

In [27]:
# Create a new model with GLPK solver
model = Model(GLPK.Optimizer)

@variable(model, cap_bid)
@variable(model, bin_var[1:in_sample_n][1:60], Bin)

@constraint(model, cap_bid >= 0)

for scenario in 1:in_sample_n
    for minute in 1:60
        @constraint(model, cap_bid - scenarios_df[minute, scenario] <= M* bin_var[scenario, minute])
    end
end
@constraint(model, sum(bin_var[scenario, minute] for scenario in 1:in_sample_n, minute in 1:60) <= q)


# Define the objective function
@objective(model, Max, cap_bid)


# Solve the model
optimize!(model)

# Check the status of the solution
status = termination_status(model)
if status == MOI.OPTIMAL
    println("Optimal solution found")
    println("Optimal objective value: ", objective_value(model))
elseif status == MOI.INFEASIBLE
    println("No feasible solution found")
else
    println("Solver status: ", status)
end


BoundsError: BoundsError: attempt to access 60-element Vector{VariableRef} at index [1, 2]